In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from conversion import images_to_volume

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [3]:
classifier_setting = 2

In [4]:
downscale = 32
voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downscale)

In [4]:
def load_scoremap_worker(stack, sec, structure, downscale, classifier_setting):
    try:
        actual_setting = resolve_actual_setting(setting=classifier_setting, stack=stack, sec=sec)
        sm = DataManager.load_scoremap(stack=stack, section=sec, structure=structure, downscale=downscale, setting=actual_setting)
        return sm
    except:
        pass

In [5]:
def load_scoremaps_multiple_sections_parallel(sections, stack, structure, downscale, classifier_setting):
    pool = Pool(12)
    scoremaps = pool.map(lambda sec: load_scoremap_worker(stack, sec, structure, downscale, classifier_setting=classifier_setting),
                                     sections)
    pool.close()
    pool.join()
    return {sec: sm for sec, sm in zip(sections, scoremaps) if sm is not None}

In [6]:
def load_scoremaps_multiple_sections_sequential(sections, stack, structure, downscale, setting):
    scoremaps = {}
    for sec in sections:
#         t = time.time()
        scoremaps[sec-1] = DataManager.load_scoremap(stack=stack, section=sec, structure=structure, downscale=downscale, setting=setting)
#         sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t))
    return scoremaps

In [9]:
for stack in ['MD603']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for structure in all_known_structures:
    for structure in ['7N']:
        print structure
    
        t = time.time()
        scoremaps = load_scoremaps_multiple_sections_parallel(stack=stack, sections=range(first_sec, last_sec+1), 
                                                        structure=structure, downscale=downscale, classifier_setting=classifier_setting)
        
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), 
                                                           first_sec=first_sec-1, last_sec=last_sec-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 2s
        
#         t = time.time()

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, structure=structure, classifier_setting=classifier_setting)
        create_if_not_exists(os.path.dirname(score_volume_filepath))
        
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)

        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale, structure=structure,
                                                                               classifier_setting=classifier_setting)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None], fmt='%d')
        
        del score_volume, scoremaps
        
#         sys.stderr.write('Save score volume: %.2f seconds\n' % (time.time() - t)) # 1s (down=32)

7N


Load scoremaps: 13.46 seconds
Create score volume: 4.68 seconds


# Alternating

In [22]:
for stack in ['MD642']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    nissl_sections = []
    for sec in range(first_sec, last_sec):
        fn = metadata_cache['sections_to_filenames'][stack][sec]
        if not is_invalid(fn) and fn.split('-')[1][0] == 'N':
            nissl_sections.append(sec)
    
    for structure in all_known_structures:
#     for structure in ['7N']:
        
        print structure
    
        t = time.time()
        scoremaps = load_scoremaps_multiple_sections_parallel(stack=stack, sections=nissl_sections, 
                                                              structure=structure, downscale=downscale, 
                                                              classifier_setting=classifier_setting)
        
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), 
                                                           first_sec=np.min(nissl_sections)-1, 
                                                           last_sec=np.max(nissl_sections)-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 2s
        
#         t = time.time()

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                      structure=structure, 
                                                                      classifier_setting=classifier_setting)
        create_if_not_exists(os.path.dirname(score_volume_filepath))
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)

#         score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale, structure=structure,
#                                                                                classifier_setting=classifier_setting)
#         np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None], fmt='%d')
        
        del score_volume, scoremaps
        
#         sys.stderr.write('Save score volume: %.2f seconds\n' % (time.time() - t)) # 1s (down=32)

5N


# Read

In [25]:
for structure in all_known_structures:
    print structure
    try:
        score_volume = DataManager.load_score_volume(stack='MD642', structure=structure, downscale=32, 
                                                 classifier_setting=classifier_setting)
        score_volume_cropped = score_volume[:, :600, :]

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                  structure=structure, 
                                                                  classifier_setting=classifier_setting)
        create_if_not_exists(os.path.dirname(score_volume_filepath))
        bp.pack_ndarray_file(score_volume_cropped.astype(np.float16), score_volume_filepath)
    except:
        pass

5N
6N
7N
7n
Amb
LC
LRt
Pn
Tz
VLL
RMC
SNC
SNR
3N
4N
Sp5I
Sp5O
Sp5C
PBG
10N
VCA
VCP
DC
AP
12N
RtTg
sp5
outerContour
SC


File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_sp5.bp
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_outerContour.bp


IC


In [ ]:
display_volume_sections(score_volume_cropped, cmap=plt.cm.hot, every=10)